In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

# import

In [2]:
# export
import torchvision

In [3]:
# export
from torch import nn

In [4]:
# export
import torch

In [5]:
# export
import torchvision

In [6]:
# export
from IPython.core import debugger as idb

In [7]:
# export
from torchvision.models.resnet import conv1x1

In [8]:
# export
from FLAI.detect_symbol.exp import databunch
from FLAI.detect_symbol.exp import resnet_ssd as resnet_ssd_detsym

# functions

### ssd_block

In [9]:
# export
class ssd_block(nn.Module):
    '''
    和detect_symbol里面的ssd_block相比只是去掉了宽高相关的部分
    '''
    def __init__(self, k, nin, n_clas):
        '''
        ssd头模块，它根据某层的特征图给出bbox预测信息，该模块的输出包含4个部分：
        -- loc：bbox中心偏移，2个值
        -- conf：目标信心，1个值
        -- clas：目标类别，n_clas个值
        ----------------------------------------
        参数：
        -- k：每个grid的anchor数
        -- nin：输入特征图通道数
        -- n_clas：目标类别数
        '''
        super().__init__()
        self.k = k
        self.oconv_loc = nn.Conv2d(nin, 2*k, 3, padding=1) # bbox center
        self.oconv_conf = nn.Conv2d(nin, 1*k, 3, padding=1) # confidence
        self.oconv_clas = nn.Conv2d(nin, n_clas*k, 3, padding=1) # classification
        
    def forward(self, x):
        return (resnet_ssd_detsym.flatten_grid_anchor(self.oconv_loc(x), self.k),
                resnet_ssd_detsym.flatten_grid_anchor(self.oconv_conf(x), self.k),
                resnet_ssd_detsym.flatten_grid_anchor(self.oconv_clas(x), self.k)
               )

### ResNetIsh_SSD

In [10]:
# export
class ResNetIsh_SSD(resnet_ssd_detsym.ResNetIsh_SSD):    
    def forward(self, x):
        outs = self._forward_impl(x)
        
        locs,confs,clss = [],[],[]
        for out in outs:
            locs += [out[0]]
            confs += [out[1]]
            clss += [out[2]]
        
        return (torch.cat(locs,dim=1),
                torch.cat(confs,dim=1),
                torch.cat(clss,dim=1)
               )

### ResNetIsh_1SSD

In [11]:
# export
class ResNetIsh_1SSD(resnet_ssd_detsym.ResNetIsh_1SSD):    
    def forward(self, x):
        outs = self._forward_impl(x)
        
        locs,confs,clss = [],[],[]
        for out in outs:
            locs += [out[0]]
            confs += [out[1]]
            clss += [out[2]]
        
        return (torch.cat(locs,dim=1),
                torch.cat(confs,dim=1),
                torch.cat(clss,dim=1)
               )

## test

In [12]:
data = databunch.get_databunch(data_root='../detect_symbol/data/ds_20200429/', cache=False)
x,y = data.one_batch(denorm=False)

num_classes = len(data.train_ds.y.classes)-1

### ResNetIsh_SSD

In [13]:
# 构建模型
m = ResNetIsh_SSD(block=torchvision.models.resnet.BasicBlock,
                  layers=[2,2,2],
                  chs=[64,128,256],
                  strides=[1,2,2],
                  pred_layerIds=[2],
                  num_anchors=[1],
                  pred_block=ssd_block,
                  num_classes=16)

In [14]:
# 跑一个batch
pred = m(x)

In [15]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
#print(49*49*4+25*25*3+13*13*3)
print(49*49*1)

type(pred)=<class 'tuple'>
len(pred)=3
-----------------------
torch.Size([64, 2401, 2])
torch.Size([64, 2401, 1])
torch.Size([64, 2401, 16])
-----------------------
2401


### ResNetIsh_1SSD

In [16]:
num_classes

17

In [17]:
# 构建模型。不涉及到hw的事情，去掉后面的层
m = ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[2,2,2],
                   chs=[64,128,256],
                   strides=[1,2,2],
                   pred_layerIds=[2],
                   num_anchors=1,
                   neck_block=resnet_ssd_detsym.cnv1x1_bn_relu,
                   head_chin=256,
                   head_block=ssd_block,
                   num_classes=17)

In [18]:
# 跑一个batch
pred = m(x)

In [19]:
# 查看输出的形状
print(f'type(pred)={type(pred)}')
print(f'len(pred)={len(pred)}')

print('-----------------------')
for p in pred:
    print(p.shape)
    
print('-----------------------')
#print(49*49*4+25*25*4+13*13*4)
print(49*49*1)

type(pred)=<class 'tuple'>
len(pred)=3
-----------------------
torch.Size([64, 2401, 2])
torch.Size([64, 2401, 1])
torch.Size([64, 2401, 17])
-----------------------
2401


## zip as function

In [26]:
# export
def get_resnet18_1ssd(layers4fpn = False, num_classes = 1):
    #layers4fpn是否保留后面的两层给fpn用
    if not layers4fpn:
        return ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[2,2,2],
                   chs=[64,128,256],
                   strides=[1,2,2],
                   pred_layerIds=[2],
                   num_anchors=1,
                   neck_block=resnet_ssd_detsym.cnv1x1_bn_relu,
                   head_chin=256,
                   head_block=ssd_block,
                   num_classes=num_classes)
    else:
        return ResNetIsh_1SSD(block=torchvision.models.resnet.BasicBlock,
                   layers=[2,2,2,2,2],
                   chs=[64,128,256,512,1024],
                   strides=[1,2,2,2,2],
                   pred_layerIds=[2],
                   num_anchors=1,
                   neck_block=resnet_ssd_detsym.cnv1x1_bn_relu,
                   head_chin=256,
                   head_block=ssd_block,
                   num_classes=num_classes)

In [27]:
# export
def get_resnet18_ssd(layers4fpn = False, num_classes = 1):
    if not layers4fpn:
        return ResNetIsh_SSD(block=torchvision.models.resnet.BasicBlock,
                  layers=[2,2,2],
                  chs=[64,128,256],
                  strides=[1,2,2],
                  pred_layerIds=[2],
                  num_anchors=[1],
                  pred_block=ssd_block,
                  num_classes=num_classes)
    else:
        return ResNetIsh_SSD(block=torchvision.models.resnet.BasicBlock,
                  layers=[2,2,2,2,2],
                  chs=[64,128,256,512,1024],
                  strides=[1,2,2,2,2],
                  pred_layerIds=[2],
                  num_anchors=[1],
                  pred_block=ssd_block,
                  num_classes=num_classes)

In [ ]:
get_resnet18_1ssd(layers4fpn = True)

In [ ]:
get_resnet18_1ssd(layers4fpn = False)

In [ ]:
get_resnet18_ssd(layers4fpn = True)

In [ ]:
get_resnet18_ssd(layers4fpn = False)

# export

In [3]:
!python ../notebook2script.py --fname 'resnet_ssd.ipynb' --outputDir '../exp/'

Converted resnet_ssd.ipynb to ../exp/resnet_ssd.py
